Let's consider how a parallel running scheme ought to work

In [13]:
from importlib import reload
import os
import GagePreprocess as gp
import numpy as np
import pickle as pkl
import time

reload(gp)

<module 'GagePreprocess' from '/Users/taixiang/Documents/UltracoldRepos/DataManagement2023/Newest/GagePreprocess.py'>

In [14]:
# given two pkl file paths, combine them into a single pkl file
def pkl_combiner(pkl1, pkl2):
    # load the pkl files, data are arrays
    data1 = gp.load_pkl(pkl1)
    data2 = gp.load_pkl(pkl2)
    # combine the data
    data = np.concatenate((data1, data2), axis=0)
    # repickle the data
    with open(pkl2, 'wb') as f:
        pkl.dump(data, f)
    

In [15]:
def pkl_deleter(pkl_files, dir_path):
    if len(pkl_files) >= 3:
        # delete the one with the earliest creation time
        ctimes = [os.path.getctime(os.path.join(dir_path, name)) for name in pkl_files]
        min_ctime_idx = ctimes.index(min(ctimes))
        os.remove(os.path.join(dir_path, pkl_files[min_ctime_idx]))
        print("Old pkl file removed")

In [16]:
def cycle_process(save_path, file_incr, jkam_path = 'High NA Imaging testenv', gage_path = 'gage testenv'):
    curr_files = []
    new_files = []
    curr_cmplx_path = ''
    curr_time_path = ''
    first_step = True
    file_num = file_incr
    curr_time_pkl = ''
    curr_cmplx_pkl = ''
    former_time_pkl = ''
    former_cmplx_pkl = ''
    curr_cmplx_arr = []
    curr_time_arr = []

    # get num files in folder
    total_files = len([name for name in os.listdir(gage_path) if os.path.isfile(os.path.join(gage_path, name))])
    if total_files >= file_num and first_step:
        # store all file names
        curr_files = [name for name in os.listdir(gage_path) if os.path.isfile(os.path.join(gage_path, name))]
        new_files = curr_files

        # run preprocessor on new files
        curr_amp_path, curr_time_path, curr_amp, curr_time = gp.process_gage_pkl(out_path = save_path, 
                                                                run_name = 'run0', 
                                                                num_shots_manual = 286,
                                                                num_frames = 3,
                                                                jkam_path = jkam_path,
                                                                gage_path = gage_path,
                                                                jkam_prefix = 'jkam_capture',
                                                                gage_prefix = 'gage_shot')
        # we also need to output filepath of pkl files
        first_step = False
        file_num += file_incr

    elif total_files >= file_num:
        curr_files = [name for name in os.listdir(gage_path) if os.path.isfile(os.path.join(gage_path, name))]
        new_files = [x for x in curr_files if x not in new_files]
        former_time_pkl = curr_time_pkl
        former_cmplx_pkl = curr_cmplx_pkl   
        # run preprocessor on new files
        curr_amp_path, curr_time_path, curr_amp, curr_time = gp.process_gage_pkl(out_path = save_path, 
                                                                run_name = 'run0', 
                                                                num_shots_manual = 286,
                                                                num_frames = 3,
                                                                jkam_path = jkam_path,
                                                                gage_path = gage_path,
                                                                jkam_prefix = 'jkam_capture',
                                                                gage_prefix = 'gage_shot')
        # combine new and old pkl files
        pkl_combiner(former_time_pkl, curr_time_path)
        pkl_combiner(former_cmplx_pkl, curr_cmplx_path)

        dir_path = os.path.dirname(former_time_pkl)
        # get if there are three files that are .pkl in this directory
        amp_pkl_files = [name for name in os.listdir(dir_path) 
                    if os.path.isfile(os.path.join(dir_path, name)) and name.endswith('.pkl') and 'cmplx_amp' in name]
        time_pkl_files = [name for name in os.listdir(dir_path) 
                    if os.path.isfile(os.path.join(dir_path, name)) and name.endswith('.pkl') and 'time' in name]
        
        pkl_deleter(amp_pkl_files, dir_path)
        pkl_deleter(time_pkl_files, dir_path)

In [17]:
def main(file_incr):
    while True:
        time.sleep(2)
        cycle_process('out', file_incr)
        print("cycled once")